In [ ]:
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import albumentations as albu
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import random
from scipy.stats import mode

import efficientnet_pytorch
from efficientnet_pytorch import EfficientNet
from torchvision import models

from utils.trainer import NNTrainer
from utils.data import MyDataset, get_train_transforms, get_valid_transforms
from utils.models import EffNetMSD

In [ ]:
data_path = './add_data/train/cbb/'
models_path = './models_pl/'
moda = True
seed = 0

models = [#['efficientnet-b5', f'{models_path}/eff5_2/'],
          #['efficientnet-b4', f'{models_path}/eff4_1/'],
          ['efficientnet-b3', f'{models_path}/eff3_6/']]

In [ ]:
cfg = {'advprop' : False,
       'img_size' : 512,
       'batch_size' : 4,
       'n_dropouts' : 30,
       'tta' : 3,
       'random_state' : 123,
       'device' : 'cuda',
       
}
cfg['augmentations'] = get_train_transforms

In [ ]:
random.seed(seed)

In [ ]:
sub = pd.DataFrame(os.listdir(data_path), columns=['image_id'])

In [ ]:
dataset = MyDataset(path_extra=data_path, extra_df=sub, img_size=cfg['img_size'],\
                              augmentations=cfg['augmentations'](cfg['img_size']),\
                              advprop=cfg['advprop'], tta=cfg['tta'], two_samples=False)

dataloader = DataLoader(dataset, batch_size=cfg['batch_size'], drop_last=False, shuffle=False)

In [ ]:
preds = []
pred_probas = None
for model_name, path in models:
    model = EffNetMSD(net=model_name, advprop=cfg['advprop'], n_dropouts=cfg['n_dropouts'])
    model_weights_files = [f'{path}/{x}' for x in os.listdir(path)]
    for model_weights_file in model_weights_files:
        model.load_state_dict(torch.load(model_weights_file, map_location=cfg['device']))
        trainer = NNTrainer(model, device=cfg['device'], label_names=[0, 1, 2, 3, 4],\
                            model_name=model_name)
        pred, pred_proba = trainer.test(dataloader, tta_moda=True,\
                                        show_results=True, verbose=True, log=False)
        if pred_probas is None:
            pred_probas = pred_proba
        else:
            pred_probas += pred_proba
        preds.append(pred)
        
        del trainer
        torch.cuda.empty_cache()
        
    del model
    torch.cuda.empty_cache()
        
pred_probas /= len(preds)

In [ ]:
if moda:
    sub['label'] = mode(np.hstack(preds), axis=1).mode.astype(int)
else:
    sub['label'] = pred_probas.argmax(axis=1)

In [ ]:
sub.to_csv('submission.csv', index=False)